# 编程范式

[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/docs/mindspore/source_zh_cn/design/programming_paradigm.ipynb)

编程范式(Programming paradigm)是指编程语言的编程风格或编程方式。通常情况下，AI框架均依赖前端编程接口所使用的编程语言的编程范式进行神经网络的构造和训练。MindSpore作为AI+科学计算融合计算框架，分别面向AI、科学计算场景，提供了面向对象编程和函数式编程的支持。同时为提升框架使用的灵活性和易用性，提出了函数式+面向对象融合编程范式，有效地体现了函数式自动微分机制的优势。

下面分别介绍MindSpore支持的三类编程范式及其简单示例。

## 面向对象编程

面向对象编程(Object-oriented programming, OOP)，是指一种将程序分解为封装数据及相关操作的模块(类)而进行的编程方式，对象为类(class)的实例。面向对象编程将对象作为程序的基本单元，将程序和数据封装其中，以提高软件的重用性、灵活性和扩展性，对象里的程序可以访问及经常修改对象相关联的数据。

在一般的编程场景中，代码(code)和数据(data)是两个核心构成部分。面向对象编程是针对特定对象(Object)来设计数据结构，定义类(Class)。类通常由以下两部分构成，分别对应了code和data：

- 方法(Methods)
- 属性(Attributes)

对于同一个Class实例化(instantiation)后得到的不同对象而言，方法和属性相同，不同的是属性的值。不同的属性值决定了对象的内部状态，因此OOP能够很好地进行状态管理。

下面为Python构造简单类的示例：

```python
class Sample: #class declaration
    def __init__(self, name): # class constructor (code)
        self.name = name # attribute (data)

    def set_name(self, name): # method declaration (code)
        self.name = name # method implementation (code)
```

对于构造神经网络来说，首要的组件就是网络层(Layer)，一个神经网络层包含以下部分：

- Tensor操作 (Operation)
- 权重 (Weights)

此二者恰好与类的Methods和Attributes一一对应，同时权重本身就是神经网络层的内部状态，因此使用类来构造Layer天然符合其定义。此外，我们在编程时希望使用神经网络层进行堆叠，构造深度神经网络，使用OOP编程可以很容易地通过Layer对象组合构造新的Layer类。

下面为使用MindSpore构造神经网络类的示例：

```python
from mindspore import nn, Parameter
from mindspore.common.initializer import initializer

class Linear(nn.Cell):
    def __init__(self, in_features, out_features, has_bias): # class constructor (code)
        super().__init__()
        self.weight = Parameter(initializer('normal', [out_features, in_features], mindspore.float32), 'weight') # layer weight (data)
        self.bias = Parameter(initializer('zeros', [out_features], mindspore.float32), 'bias') # layer weight (data)

    def construct(self, inputs): # method declaration (code)
        output = ops.matmul(inputs, self.weight.transpose(0, 1)) # tensor transformation (code)
        output = output + self.bias # tensor transformation (code)
        return output
```

## 函数式编程

函数式编程(Functional programming)是一种将计算机运算视为函数运算，并且避免使用程序状态以及可变对象的编程范式。

在函数式编程中，函数被视为一等公民，这意味着它们可以绑定到名称（包括本地标识符），作为参数传递，并从其他函数返回，就像任何其他数据类型一样。这允许以声明性和可组合的风格编写程序，其中小功能以模块化方式组合。函数式编程有时被视为纯函数式编程的同义词，是将所有函数视为确定性数学函数或纯函数的函数式编程的一个子集。当使用一些给定参数调用纯函数时，它将始终返回相同的结果，并且不受任何可变状态或其他副作用的影响。

函数式编程有两个核心特点，使其十分符合科学计算的需要：

1. 编程函数语义与数学函数语义完全对等。
2. 确定性，给定相同输入必然返回相同输出。无副作用。

由于确定性这一特点，通过限制副作用，程序可以有更少的错误，更容易调试和测试，更适合形式验证。

MindSpore提供纯函数式编程的支持，配合`mindspore.numpy`及`mindspore.scipy`提供的数值计算接口，可以便捷地进行科学计算编程。下面是使用函数式编程的示例：

```python
import mindspore.numpy as mnp
from mindspore import grad

grad_tanh = grad(mnp.tanh)
print(grad_tanh(2.0))
# 0.070650816

print(grad(grad(mnp.tanh))(2.0))
print(grad(grad(grad(mnp.tanh)))(2.0))
# -0.13621868
# 0.25265405
```

## 函数式+面向对象融合编程

考虑到神经网络模型构建和训练流程的灵活性和易用性需求，结合MindSpore自身的函数式自动微分机制，MindSpore针对AI模型训练设计了函数式+面向对象融合编程范式，可以兼顾面向对象编程和函数式编程的优势，同时使用同一套自动微分机制实现深度学习反向传播和科学计算自动微分的兼容，从底层支持AI和科学计算建模的兼容。下面是函数式+面向对象融合编程的典型过程：

1. 用类构建神经网络；
2. 实例化神经网络对象；
3. 构造正向函数，连接神经网络和损失函数；
4. 使用函数变换，获得梯度计算（反向传播）函数；
5. 构造训练过程函数；
6. 调用函数进行训练。

下面是函数式+面向对象融合编程的简单示例：

```python
# Class definition
class Net(nn.Cell):
    def __init__(self):
        ......
    def construct(self, inputs):
        ......

# Object instantiation
net = Net() # network
loss_fn = nn.CrossEntropyLoss() # loss function
optimizer = nn.Adam(net.trainable_params(), lr) # optimizer

# define forward function
def forword_fn(inputs, targets):
    logits = net(inputs)
    loss = loss_fn(logits, targets)
    return loss, logits

# get grad function
grad_fn = value_and_grad(forward_fn, None, optim.parameters, has_aux=True)

# define train step function
def train_step(inputs, targets):
    (loss, logits), grads = grad_fn(inputs, targets) # get values and gradients
    optimizer(grads) # update gradient
    return loss, logits

for i in range(epochs):
    for inputs, targets in dataset():
        loss = train_step(inputs, targets)
```

如上述示例，在神经网络构造时，使用面向对象编程，神经网络层的构造方式符合AI编程的习惯。在进行前向计算和反向传播时，MindSpore使用函数式编程，将前向计算构造为函数，然后通过函数变换，获得`grad_fn`，最后通过执行`grad_fn`获得权重对应的梯度。

通过函数式+面向对象融合编程，即保证了神经网络构建的易用性，同时提高了前向计算和反向传播等训练过程的灵活性，是MindSpore推荐的默认编程范式。